# Setup
Update path, load assemblies and import modules

In [ ]:
import os
import sys
sys.path.extend([
    '.venv/Lib',
    '.venv/Lib/site-packages',
    '../LibSLH/bin/Release/net48'])
import clr
sys.path.append('')

clr.AddReference('System.Drawing')
clr.AddReference('System.Net')
clr.AddReference('LibreMetaverse')
clr.AddReference('LibreMetaverse.StructuredData')
clr.AddReference('LibSLH')

# Commonly used
from OpenMetaverse import *
from LibSLH import *

# Important imports
from slhstudio import *
from helpers import *

# Used for specific examples
import System.Drawing.Imaging
from System.Net import HttpWebRequest
from OpenMetaverse.Http import CapsBase
from OpenMetaverse.StructuredData import *
import math
import asyncio
from IPython.display import display, display_html, clear_output
from ipywidgets import widgets
from collections import defaultdict
from types import SimpleNamespace
from base64 import b64decode
import zlib

# Start the Client
Can also be used to restart a new client instance

In [ ]:
client = Client()

client_namespace = SimpleNamespace()
def client_event_handler(event):
    return event_handler(client_namespace, event)

client.start()

# Spin to Load
Make the client's camera rotate in place. This ensures that world objects will begin to load for the client as the sim provides updates.

In [ ]:
def spin_camera(client):
    while True:
        for i in range(16):
            theta = math.pi * i / 16
            direction = Vector3(math.cos(theta), math.sin(theta), 0.0)
            # Oof gamer https://github.com/pythonnet/pythonnet/issues/906
            rotation = Vector3.RotationBetween(Vector3.UnitX, Vector3.Normalize(direction))
            target = Vector3.Add(client.Self.SimPosition, direction)  
            client.Self.Movement.Camera.LookAt(client.Self.SimPosition, target);
            client.Self.Movement.SendUpdate()
            yield asyncio.sleep(0.1)

client.start_coroutine(spin_camera, "spin_camera")

# Materials Look-up Table
Periodically request the sim for materials data.

In [ ]:
materials_lut = dict()

def setup_request(address, method, accept_header, content=None):
        req = HttpWebRequest.Create(address)
        req.Method = method
        req.Accept = accept_header
        req.ServicePoint.MaxIdleTime = 0
        req.ServicePoint.Expect100Continue = False
        if content:
            stream = req.GetRequestStream()
            stream.Write(content)
            stream.Close()
            req.ContentLength = len(content);
        return req

def osd_to_dict(osd):
    d = {k: v for k, v in zip(osd.Keys, osd.Values)}
    for k, v in d.items():
        if isinstance(v, OSDMap):
            d[k] = osd_to_dict(v)
        elif isinstance(v, OSDInteger):
            d[k] = v.AsInteger()
        elif isinstance(v, OSDReal):
            d[k] = v.AsReal()
        elif isinstance(v, OSDUUID):
            d[k] = v.AsUUID()
    return d

def binary_to_uuid(osd):
    return UUID(osd.AsBinary(), 0)
    
def request_materials(client):
    address = client.Network.CurrentSim.Caps.CapabilityURI("RenderMaterials")
    req = setup_request(address, "GET", "application/llsd+xml")
    
    def progress_callback(request, response, bytes_received, total_bytes_to_receive):
        pass
    
    def complete_callback(request, response, response_data, error):
        #print(request, response, bytes(response_data), error)
        sd = OSDParser.DeserializeLLSDXml(response_data)
        b64_zipped = sd["Zipped"].AsString()
        zipped = b64decode(b64_zipped)
        unzipped = zlib.decompress(zipped)
        materials_sd = OSDParser.DeserializeLLSDBinary(unzipped)
        materials = {binary_to_uuid(x.Value): osd_to_dict(y.Value) for x, y in list(materials_sd)}
        materials_lut.update(materials)
    
    CapsBase.DownloadDataAsync(
        req,
        client.Settings.CAPS_TIMEOUT,
        CapsBase.DownloadProgressEventHandler(progress_callback),
        CapsBase.RequestCompletedEventHandler(complete_callback))


def update_materials(client):
    while True:
        request_materials(client)
        yield asyncio.sleep(5)
        
client.start_coroutine(update_materials, "update_materials")

# Examples

### Say a message in local chat

In [ ]:
def say(obj):
    client.Self.Chat(str(obj), 0, ChatType.Normal)

In [ ]:
say([x for x in range(10)])

### Misc. examples

In [ ]:
avatars = {str(e.Key): str(e.Value) for e in client.Network.CurrentSim.AvatarPositions.Copy()}
say(avatars)
print(avatars)

In [ ]:
{str(e.Key): str(e.Value) for e in client.Friends.FriendList.Copy()}

In [ ]:
def debug_point_at(sender, args):
    pt = str(args.PointType)
    id = str(args.TargetID)
    pos = str(args.TargetPosition)
    message = f'{pt}, {id}, {pos}'
    client.Self.Chat(message, 0, ChatType.Normal)
remove_all_event_handlers(client.Avatars, 'ViewerEffectPointAt')
client.Avatars.ViewerEffectPointAt += debug_point_at

### Accept teleport requests

In [ ]:
# TODO
authorized_avatars = {}
authorized_av_tf = widgets.Text()
display(authorized_av_tf)

def authorized_av_tf_submit(sender):
    avatars = sender.value.split(',')
    query_ids = [UUID.Random() for _ in avatars]
    lookup = dict(zip(query_ids, avatars))
    def handle_reply(sender, args):
        # args.Avatars seems to always have just 1 element
        if args.QueryID in lookup:
            info = list(args.Avatars)[0]
            uuid, name = info.Key, info.Value
            authorized_avatars[uuid] = name
            del lookup[args.QueryID]
        if not lookup:
            print('done')
            client.Avatars.AvatarPickerReply -= handle_reply
        
    
    remove_all_event_handlers(client.Avatars, 'AvatarPickerReply')
    client.Avatars.AvatarPickerReply += handle_reply
    for qid in lookup:
        client.Avatars.RequestAvatarNameSearch(lookup[qid], qid)

@client_event_handler(client.Self.IM)
def handle_instant_message(sender, args):
    dialog = args.IM.Dialog
    if dialog == 22: # RequestTeleport
        from_agent_id = args.IM.FromAgentID
        if from_agent_id in authorized_avatars:
            session_id = args.IM.IMSessionID
            client.Self.TeleportLureRespond(from_agent_id, session_id, True)
    

authorized_av_tf.on_submit(authorized_av_tf_submit)

### Debug Textures!

In [ ]:
def get_prim_texture_info(prim):
    yield prim.Textures.DefaultTexture
    yield from filter(lambda t: t, prim.Textures.FaceTextures)

def display_multi_image(paths, max_width=256):
    tags = [f'<img src="{path}" style="display: inline-block; max-width: {max_width}px">' for path in paths]
    display_html('\n'.join(tags), raw=True)

def get_image_path(uuid):
    path = os.path.join('img_cache', str(uuid) + '.png')
    if not os.path.exists(path):
        client.GetTextureByUUID(uuid).Save(path)
    return path


@client_event_handler(client.Avatars.ViewerEffectPointAt)
def handle_viewer_effect_point_at(sender, args):
    if args.PointType == PointAtType.Select:
        local_id = client.GetPrimLocalId(args.TargetID)
        parent_id = client.GetParentLocalId(local_id)
        parent_id = parent_id or local_id
        link_set = list(client.GetLinkSetLocalIds(parent_id))
        prims = [client.Objects.GetPrimitive(args.Simulator, System.UInt32(id), UUID.Zero, False)
                 for id in link_set]
        tex_info = [info for prim in prims for info in get_prim_texture_info(prim)]
        tex_uuids = {info.TextureID for info in tex_info}
        mat_uuids = {info.MaterialID for info in tex_info}
        try:
            for mat_uuid in mat_uuids:
                if mat_uuid in materials_lut:
                    normal_tex = materials_lut[mat_uuid]['NormMap']
                    shiny_tex = materials_lut[mat_uuid]['SpecMap']
                    if normal_tex != UUID.Zero:
                        tex_uuids.add(normal_tex)
                    if shiny_tex != UUID.Zero:
                        tex_uuids.add(shiny_tex)
        except Exception as e:
            print(e)
            raise e
        clear_output()
        print("Loading images, please wait...")
        file_paths = [get_image_path(uuid) for uuid in tex_uuids]
        clear_output()
        display_multi_image(file_paths, 64)
        for material_id in mat_uuids:
            request_material(material_id)

In [ ]:
bar = 0
def foo(s, a):
    global bar
    print(bar)
    bar += 1

In [ ]:
client._coroutines

# Stop the client

In [ ]:
client.stop()